# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [2]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files('data/song_data/')

In [6]:
filepath = song_files[0]

In [7]:
song_df = pd.DataFrame(pd.read_json(filepath, lines=True,orient='columns'))
song_df

artist_id  artist_latitude  artist_location  artist_longitude  \
0  ARD7TVE1187B99BFB1              NaN  California - LA               NaN   

  artist_name   duration  num_songs             song_id             title  \
0      Casual  218.93179          1  SOMZWCG12A8C13C480  I Didn't Mean To   

   year  
0     0

In [8]:
song_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 10 columns):
artist_id           1 non-null object
artist_latitude     0 non-null float64
artist_location     1 non-null object
artist_longitude    0 non-null float64
artist_name         1 non-null object
duration            1 non-null float64
num_songs           1 non-null int64
song_id             1 non-null object
title               1 non-null object
year                1 non-null int64
dtypes: float64(3), int64(2), object(5)
memory usage: 160.0+ bytes


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [9]:
# Extract data
song_data = song_df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()

In [10]:
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [11]:
#insert into table
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [12]:
artist_data = song_df[['artist_id',\
                  'artist_name',\
                  'artist_location',\
                  'artist_latitude',\
                  'artist_longitude']].values[0].tolist()
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [13]:
#insert into table
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [14]:
log_files = get_files('data/log_data/')

In [15]:
filepath = log_files[0]

In [16]:
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_json.html
log_df = pd.read_json(filepath, lines=True, orient='columns')
log_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 18 columns):
artist           330 non-null object
auth             388 non-null object
firstName        376 non-null object
gender           376 non-null object
itemInSession    388 non-null int64
lastName         376 non-null object
length           330 non-null float64
level            388 non-null object
location         376 non-null object
method           388 non-null object
page             388 non-null object
registration     376 non-null float64
sessionId        388 non-null int64
song             330 non-null object
status           388 non-null int64
ts               388 non-null int64
userAgent        376 non-null object
userId           388 non-null object
dtypes: float64(2), int64(4), object(12)
memory usage: 54.6+ KB


In [17]:
log_df.head(3)

artist       auth firstName gender  itemInSession lastName  \
0  Stephen Lynch  Logged In    Jayden      M              0     Bell   
1        Manowar  Logged In     Jacob      M              0    Klein   
2      Morcheeba  Logged In     Jacob      M              1    Klein   

      length level                             location method      page  \
0  182.85669  free      Dallas-Fort Worth-Arlington, TX    PUT  NextSong   
1  247.56200  paid  Tampa-St. Petersburg-Clearwater, FL    PUT  NextSong   
2  257.41016  paid  Tampa-St. Petersburg-Clearwater, FL    PUT  NextSong   

   registration  sessionId                                  song  status  \
0  1.540992e+12        829                     Jim Henson's Dead     200   
1  1.540558e+12       1049                           Shell Shock     200   
2  1.540558e+12       1049  Women Lose Weight (Feat: Slick Rick)     200   

              ts                                          userAgent userId  
0  1543537327796  Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...     91  
1  1543540121796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     73  
2  1543540368796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     73

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [18]:
#filter by NextSong
df = log_df[log_df['page'] == 'NextSong']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330 entries, 0 to 387
Data columns (total 18 columns):
artist           330 non-null object
auth             330 non-null object
firstName        330 non-null object
gender           330 non-null object
itemInSession    330 non-null int64
lastName         330 non-null object
length           330 non-null float64
level            330 non-null object
location         330 non-null object
method           330 non-null object
page             330 non-null object
registration     330 non-null float64
sessionId        330 non-null int64
song             330 non-null object
status           330 non-null int64
ts               330 non-null int64
userAgent        330 non-null object
userId           330 non-null object
dtypes: float64(2), int64(4), object(12)
memory usage: 49.0+ KB


In [19]:
# Convert the ts timestamp column to datetime 
t = pd.to_datetime(df['ts'], unit='ms')
t.head()

0   2018-11-30 00:22:07.796
1   2018-11-30 01:08:41.796
2   2018-11-30 01:12:48.796
3   2018-11-30 01:17:05.796
4   2018-11-30 01:20:56.796
Name: ts, dtype: datetime64[ns]

In [20]:
# Extract the timestamp, hour, day, week of year, month, year, and weekday from the ts column and set time_data to a list containing these values in order
# https://www.w3resource.com/python-exercises/lambda/python-lambda-exercise-8.php

df["ts"] = pd.to_datetime(df["ts"], unit='ms')
df["hour"], df["day"], df["week"], df["month"], df["year"], df["weekday"] = \
    df["ts"].apply(lambda x: x.hour), \
    df["ts"].apply(lambda x: x.day), \
    df["ts"].apply(lambda x: x.week), \
    df["ts"].apply(lambda x: x.month), \
    df["ts"].apply(lambda x: x.year), \
    df["ts"].apply(lambda x: x.dayofweek)


time_data = (df.loc[:,["ts",\
                       "hour",\
                       "day",\
                       "week",\
                       "month",\
                       "year",\
                       "weekday"]].drop_duplicates())

time_data.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


ts  hour  day  week  month  year  weekday
0 2018-11-30 00:22:07.796     0   30    48     11  2018        4
1 2018-11-30 01:08:41.796     1   30    48     11  2018        4
2 2018-11-30 01:12:48.796     1   30    48     11  2018        4
3 2018-11-30 01:17:05.796     1   30    48     11  2018        4
4 2018-11-30 01:20:56.796     1   30    48     11  2018        4

In [22]:
time_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330 entries, 0 to 387
Data columns (total 7 columns):
ts         330 non-null datetime64[ns]
hour       330 non-null int64
day        330 non-null int64
week       330 non-null int64
month      330 non-null int64
year       330 non-null int64
weekday    330 non-null int64
dtypes: datetime64[ns](1), int64(6)
memory usage: 20.6 KB


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [23]:
# insert into table
for i, row in time_data.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [24]:
# load user table
user_data1 = df.get(['userId','firstName','lastName','gender','level'])
    
# rename_attributes to match the tables already created in sql_queries.py
user_data1.columns = ['user_id','first_name','last_name','gender','level']

# data cleanup - remove rows with no user_id
user_clean = user_data1[user_data1['user_id']!= '']

# data cleanup - remove duplicates
user_dups_remove = user_clean.drop_duplicates('user_id',keep='first')
user_df = user_dups_remove
user_df.head()

# df['userId'].value_counts()
# df[df['userId'] == 15]
# df['itemInSession'].value_counts()
#df[df['itemInSession'] == 0]; ALL ON 11.30.2018

user_id first_name last_name gender level
0       91     Jayden      Bell      M  free
1       73      Jacob     Klein      M  paid
23      86      Aiden      Hess      M  free
30      24      Layla   Griffin      F  paid
40      26       Ryan     Smith      M  free

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [25]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [28]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        song_id, artist_id = results
    else:
        song_id, artist_id = None, None

    # insert songplay record
    
    #create start_time attribute
    start_time = pd.to_datetime(row.ts,unit='ms').strftime('%Y-%m-%d %I:%M:%S')

    # insert songplay record
    songplay_data = (start_time,\
                 row.userId,\
                 row.level,\
                 str(song_id),\
                 str(artist_id),\
                 row.sessionId,\
                 row.location,\
                 row.userAgent)
    try:
        cur.execute(songplay_table_insert, songplay_data)
    except psycopg2.Error as e:
        print("Error: Inserting row for table: songplays")
        print (e)
    except psycopg2.Error as e:
        print("Error: Querying for Song ID and Artist ID")
        print (e)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

In [29]:
songplay_data.info()

AttributeError: 'tuple' object has no attribute 'info'

# Close Connection to Sparkify Database

In [27]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.